# Session 5: Going OOP

## Recap

At the HAW, your journey into the wonderful world of programming typically starts with C. As a language that dates back to the mid-60s, it follows the classic procedural programming paradigm: C code is a series of calls to functions that can again call other functions or contain more elemental statements. Every line of code changes the state of the program, which also makes C an imperative language.

Most common, recent programming languages are still imperative, but they extend the idea of procedural programming by introducing the concept of classes. They bring together variables and functions (attributes and methods in OOP slang) that act on them, serving as templates for the creation of a new kind of data structure: objects. Thanks to them, programmers are no longer limited to primitive datatypes such as integers or floats, but can work with higher-level strctures. In addition, object-oriented languages such as Java or C++ allow us to control the range of values that a given attribute can take via the ``constructor`` method that creates a new object, and protects attributes from undesired changes via access modifiers such as ``private`` or ``public``, with ``setters`` and ``getters`` as interfaces, a concept which is refered to as *encapsulation* and - to some extent - *abstraction*.

The next big pillar of object-oriented programming is the idea of *inheritance*: that is, a class that inherits the attributes and methods of another class but extends it with new attributes and methods to avoid needless code redundance. Similarily, thanks to *polymorphism* we don't have a implement new operators, methods or functions for every new object that we create, but we can reuse existing ones and overload their argument list. Some examples are the basic arithmetical operators such as ``+`` or ``-`` that can deal with various sorts of objects, or the ``len`` function. 

As an introduction to the beautiful world of object-orented python programming, we will implement a simple class ``Markov`` (see Markov chains) that will help us understand one fundamental principle of neural networks: signal feed-forward via matrices and vectors.

## Transition matrices

Matrices can be used to model transitions such as nuclear decay chains, voter currents or feed-forward of signals through a neural network ...

Here, we will discuss stochastic matrices which model transitions between a set number of nodes. Every node starts with a set quantity of whatever we wish to model, e.g. element abundances, number of voters or signal strengths. A state vector $\vec{v} = (x, y, z, ...)$ holds these quantities, where the first entry corresponds to node 1, the second entry to node 2, and so on. As these quantities evolve over time, the initial state vector can be referred to as $\vec{v}_0$, the next state vector is $\vec{v}_1$, and so on.

Given a state $\vec{v}_n$, the next state can be calculated via

$\vec{v}_{n+1} = M \cdot \vec{v}_n$

where $M$ is the transitional matrix that models the mixing and re-distribution that happens during one timestep. In a stochastic context, the matrix entries are between 0 and 1, representing fractions that shift between the nodes. 

## A first example

There has been a revolution in the state of Neurostan. Three parties are competing for votes:

- The Green House (G)
- The Central Party (C)
- The Red Faction (R)

Right after the revolution, political surveys found out that of the 10 million people of Neurostan, 10% would vote for the Green House, 80% for the Central Party and another 10% for the Red Faction, i.e. we have an initial state vector of

$\vec{v}_0 = ( 1, 8, 1 )$

representing millions of people. Through extensive surveys, demographs found the following weekly voter currents:

- Green voters vote 95% Green, 4% Central, 1% Red
- Central voters vote 10% Green, 85% Central, 5% Red
- Red voters vote 2% Green, 1% Central, 97% Red

This can be directly translated into a transition matrix:

$M = \left(\begin{array}{rrr}                                
0.95 & 0.10 & 0.02 \\                                               
0.04 & 0.85 & 0.01 \\                                               
0.01 & 0.05 & 0.97 \\                                               
\end{array}\right)$

Note that this is exactly how signals are mixed and fed forward inside a neural network from one layer to another:

![Markov chain](chain.PNG "Title")

## Implementing the ``Markov`` class

As all of you have gone through at least one semester of Java, we will start with the code snippet and focus on syntactic pecularities. A new class is created using the ``class`` keyword, followed by the class name in CapWords and an empty set of brackets. Inside the class, we can define global class attributes using regular variable initializations. Then, we need to define a class constructor using the *magic* method ``__init__``. Its argument list usually contains ``self`` which references the object instance that will be created by calling the constructor (cp. ``this`` in Java), and all the arguments that the user will have to pass to the constructor in order to create an object. ``colors`` is a list of ``matplotlib`` color strings that we will be using in a plotting method, but we set defined it as an optional argument with an empty list being the default:

```python
class Markov():
    def __init__(self, v0, M, colors = []):
        self.__i = 0
        self.__v = [v0]
        self.__M = M
        self.colors = colors
```

Within the constructor, we assign the arguments to the respective class attributes: The timestep index ``i``, the initial state vector ``v0`` as first element of a list ``v`` and the transition matrix ``M``. Note that I am using a double underscore to mark them as ``private`` attributes that may not be directly accessible, while the color list can be. Without an underscore, attributes are regarded as ``public``, so ``colors`` can be accessed and modified directly without the need of a ``setter`` method. Strictly speaking, Python just does some name mangling on the ``private`` attributes and they are still accessible if you know the syntax (``_Classname__Attribute``), but the double underscore informs anyone who is using your code that such usage was not intended.

The second method one typically implements is another magic method:

```python
    def __str__(self):
        return "Timestep: {}\nState: {}".format(self.__i, self.__v[-1])
```

It returns a string and defines how to interpret an instance of the class as string, allowing us to use simple ``print`` statements to get some information on the state of the object.

Finally, let's implement a method that evolves the system. According to theory, we need to perform a dot product between the state vector and the transition matrix, and append it to the list of vectors. At the same time, we need to increase the state index:

```python
    def evolve(self):
        self.__v.append( np.dot(M, self.__v[-1]) )
        self.__i += 1
```

Time to compile it all in a single cell as proper code:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

class Markov():
    def __init__(self, v0, M, colors = []):
        self.__i = 0
        self.__v = [v0]
        self.__M = M
        self.colors = colors
               
    def __str__(self):
        return "Timestep: {}\nState: {}".format(self.__i, self.__v[-1])
    
    def evolve(self):
        self.__v.append( np.dot(M, self.__v[-1]) )
        self.__i += 1

To create an instance of the class, i.e. an object, we need to define ``v0`` and ``M`` according to the theory part at the beginning of this notebook:

In [ ]:
v0 = np.array( [1.,8.,1.] )
M = np.array( [[0.95,0.1,0.02],[0.04,0.85,0.01],[0.01,0.05,0.97] ] )

By using the class name like a function, we automatically call the class constructor:

In [ ]:
neurostan = Markov(v0, M, ["green","blue","red"])

print(neurostan)

With the ``dir`` function we can inspect the methods and attributes that ``Markov`` inherited from the base class as well as the stuff that we have implemented:

In [ ]:
print(dir(neurostan))

Note that our *private* attributes may still be accessed by using the alias that has automatically been assigned to them just as the public attribute ``colors``:

In [ ]:
print(neurostan._Markov__v)
print(neurostan.colors)

To conclude this part, let's evolve our object a single timestep:

In [ ]:
neurostan.evolve()
print(neurostan)

Classes, functions and other code snippets can be outsourced into external modules. Just create a text file ``chain.py`` (note the file extension), paste the class ``Markov`` and save the file. Now you can use the filename to import the class into your notebook:

In [ ]:
import chain

print(type(chain))
print(dir(chain))

The class constructor and other methods now exist within the new namespace ``chain``:

In [ ]:
neurostan = chain.Markov(v0, M, ["green","blue","red"])

print(neurostan)

Naturally, you can restrict the import statement to specific parts of the module which will add them to your global namespace:

In [ ]:
from chain import Markov

neurostan = Markov(v0, M, ["green","blue","red"])

print(neurostan)